In [3]:
import pandas as pd

# Path to the downloaded file in Kaggle environment
file_path = "S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\data\\temperature-v1.csv"

# Load the CSV file into a DataFrame
data = pd.read_csv(file_path)

# Display the first few rows to understand the structure of the data
print("Before date conversion:")
print(data.head())

# Convert the 'Datetime' column to datetime
data['Datetime'] = pd.to_datetime(data['Datetime'])

# Filter the data to keep only rows from the year 2010
data = data[data['Datetime'].dt.year >= 2010]

# Format the 'Datetime' column to the desired format: %Y-%m-%d %H:%M
data['Datetime'] = data['Datetime'].dt.strftime('%Y-%m-%d %H:%M')

# Display the first few rows after filtering and conversion
print("After filtering for >=2010 and date conversion:")
print(data.head())

# Save the modified DataFrame back to a new CSV file
output_file_path = "S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\data\\temperature-v1_modified.csv"
data.to_csv(output_file_path, index=False)

# Confirm that the file has been saved
print(f"Modified file saved as {output_file_path}")

Before date conversion:
              Datetime  ABI   ACT  AMA  ALB  JFK  LGA
0  1949-01-01 00:00:00  6.1  10.6  2.2 -3.3 -3.3 -3.9
1  1949-01-01 01:00:00  3.3   6.7  1.1 -3.3 -3.3 -3.9
2  1949-01-01 02:00:00  2.2   5.6  0.6 -3.3 -3.3 -3.9
3  1949-01-01 03:00:00  1.7   4.4  0.0 -2.8 -3.3 -3.9
4  1949-01-01 04:00:00  1.7   3.9  1.1 -2.8 -3.3 -3.9
After filtering for >=2010 and date conversion:
                Datetime  ABI  ACT  AMA  ALB  JFK  LGA
470398  2010-01-01 00:00  1.1  6.7 -0.6 -1.7  1.1  1.1
470399  2010-01-01 01:00  1.1  6.1 -3.3 -1.7  1.1  1.1
470400  2010-01-01 02:00  1.7  5.6 -3.9 -1.7  0.6  1.1
470401  2010-01-01 03:00  1.1  5.0 -3.9 -1.7  0.6  1.1
470402  2010-01-01 04:00  1.1  4.4 -5.0 -1.1  0.6  0.6
Modified file saved as S:\spatiotemporal-analysis\spacetimeformer-main\spacetimeformer\data\temperature-v1_modified.csv


In [6]:
# Haversine formula to compute the distance between two points given their latitudes and longitudes
def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Function to calculate the effective radius based on area
def effective_radius(area):
    return np.sqrt(area / np.pi)

# Sample data
data = {
    'state': ['QLD', 'NSW', 'SA', 'TAS', 'VIC'],
    'lat': [-22.5752, -31.2532, -30.0002, -42.0409, -36.9848],
    'long': [144.0848, 146.9211, 136.2092, 146.8087, 143.3906],
    'area_km_sq': [1729742, 801150, 984321, 68401, 227444]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Calculate the effective radii for each state
df['effective_radius'] = df['area_km_sq'].apply(effective_radius)

# Initialize list to store both normalized and physical distances
distances = []

# Loop through pairs of states to calculate distances
for i in range(len(df)):
    for j in range(len(df)):
        # Get the coordinates, areas and radii
        lat1, lon1, r1 = df.iloc[i]['lat'], df.iloc[i]['long'], df.iloc[i]['effective_radius']
        lat2, lon2, r2 = df.iloc[j]['lat'], df.iloc[j]['long'], df.iloc[j]['effective_radius']
        
        # Calculate the physical distance between the centroids using the Haversine formula
        distance = haversine(lat1, lon1, lat2, lon2)
        
        # Calculate the normalized distance
        D_norm = distance / (r1 + r2)
        
        # Append the results to the list
        distances.append({
            'State 1': df.iloc[i]['state'],
            'State 2': df.iloc[j]['state'],
            'Physical Distance (km)': distance,
            'Normalized Distance': D_norm
        })

# Create a DataFrame with both physical and normalized distances
distances_df = pd.DataFrame(distances)

# Display the resulting DataFrame
distances_df

,State 1,State 2,Physical Distance (km),Normalized Distance
0,QLD,QLD,0.000000,0.000000
1,QLD,NSW,1004.958524,0.805895
2,QLD,SA,1138.614306,0.874667
3,QLD,TAS,2179.251774,2.449765
4,QLD,VIC,1603.659847,1.586073
5,NSW,QLD,1004.958524,0.805895
6,NSW,NSW,0.000000,0.000000
7,NSW,SA,1033.954847,0.971088
8,NSW,TAS,1199.579094,1.838309
9,NSW,VIC,715.254517,0.924033


In [8]:
distances_df[distances_df['State 1'] == 'NSW'].sort_values('Normalized Distance')

,State 1,State 2,Physical Distance (km),Normalized Distance
6,NSW,NSW,0.000000,0.000000
5,NSW,QLD,1004.958524,0.805895
9,NSW,VIC,715.254517,0.924033
7,NSW,SA,1033.954847,0.971088
8,NSW,TAS,1199.579094,1.838309


!python S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\train.py spacetimeformer asos --context_points 128 --target_points 40 --run_name spacetimeformer_asos --batch_size 32 --warmup_steps 1000 --d_model 200 --d_ff 700 --enc_layers 1 --dec_layers 1 --dropout_emb .1 --dropout_ff .3 --d_qk 30 --d_v 30 --n_heads 10 --patience 10 --decay_factor .8 --data_path S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\data\\temperature-v1_modified.csv

In [1]:
import pandas as pd

def subset_csv(src_path: str, dest_path: str, columns: list):
    """
    Reads a CSV file from src_path, extracts the specified columns, and writes the subset to dest_path.
    
    Parameters:
        src_path (str): Path to the source CSV file.
        dest_path (str): Path to save the subset CSV file.
        columns (list): List of column names to extract from the source CSV.
    """

    # Load the CSV file into a DataFrame
    df = pd.read_csv(src_path)
    
    # Create a subset of the DataFrame with the specified columns
    subset = df[columns]
    
    # Save the subset to the destination CSV file
    subset.to_csv(dest_path, index=False)
    print(f"Subset saved to {dest_path}")

In [4]:
subset_csv('/Users/sathwick/SDrive/ISU/S25/599/aemo_merged.csv', '/Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_vlc.csv', ['SETTLEMENTDATE', "TOTALDEMAND_NSW", "RRP_NSW", "TOTALDEMAND_VIC", "RRP_VIC",])
subset_csv('/Users/sathwick/SDrive/ISU/S25/599/aemo_merged.csv', '/Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_qld.csv', ['SETTLEMENTDATE', "TOTALDEMAND_NSW", "RRP_NSW", "TOTALDEMAND_QLD", "RRP_QLD",])

Subset saved to /Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_vlc.csv
Subset saved to /Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_qld.csv


In [5]:
pd.read_csv('/Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_vlc.csv')

,SETTLEMENTDATE,TOTALDEMAND_NSW,RRP_NSW,TOTALDEMAND_VIC,RRP_VIC
0,2015-02-01 00:00,6529.375000,19.210000,4485.130000,14.590000
1,2015-02-01 01:00,6145.710000,18.195000,4055.830000,12.925000
2,2015-02-01 02:00,5782.555000,16.450000,3701.360000,10.440000
3,2015-02-01 03:00,5600.055000,14.200000,3488.495000,6.660000
4,2015-02-01 04:00,5619.520000,15.955000,3455.595000,4.380000
...,...,...,...,...,...
88100,2025-02-18 20:00,7977.682500,106.371667,5107.343333,93.497500
88101,2025-02-18 21:00,7653.483333,100.095833,4648.734167,87.070833
88102,2025-02-18 22:00,7361.791667,97.980000,4341.420833,84.615000
88103,2025-02-18 23:00,7002.186667,96.224167,4404.169167,86.157500


In [6]:
pd.read_csv('/Users/sathwick/SDrive/ISU/S25/599/aemo_nsw_qld.csv')

,SETTLEMENTDATE,TOTALDEMAND_NSW,RRP_NSW,TOTALDEMAND_QLD,RRP_QLD
0,2015-02-01 00:00,6529.375000,19.210000,5687.755000,17.855000
1,2015-02-01 01:00,6145.710000,18.195000,5349.120000,16.100000
2,2015-02-01 02:00,5782.555000,16.450000,5148.620000,14.420000
3,2015-02-01 03:00,5600.055000,14.200000,5045.510000,13.025000
4,2015-02-01 04:00,5619.520000,15.955000,5066.075000,14.620000
...,...,...,...,...,...
88100,2025-02-18 20:00,7977.682500,106.371667,8026.143333,112.365000
88101,2025-02-18 21:00,7653.483333,100.095833,7663.199167,102.720833
88102,2025-02-18 22:00,7361.791667,97.980000,7187.470833,102.015000
88103,2025-02-18 23:00,7002.186667,96.224167,6736.655000,100.227500


In [2]:
import pandas as pd

def filter_columns_and_save(input_file_path, selected_columns, output_file_path):
    # Read the original dataframe from the input CSV file
    df = pd.read_csv(input_file_path)
    
    # Filter the dataframe to only include the selected columns
    df_filtered = df[selected_columns]
    
    # Save the filtered dataframe to the specified output file path
    df_filtered.to_csv(output_file_path, index=False)
    print(f"Filtered dataframe saved to {output_file_path}")

In [3]:
l = [['NSW'],
 ['TAS'],
 ['QLD'],
 ['VIC'],
 ['SA'],
 ['NSW', 'QLD'],
 ['NSW', 'VIC'],
 ['QLD', 'SA'],
 ['TAS', 'VIC'],
 ['NSW', 'SA', 'VIC'],
 ['NSW', 'QLD', 'VIC'],
 ['NSW', 'QLD', 'SA'],
 ['NSW', 'TAS', 'VIC'],
 ['NSW', 'QLD', 'SA', 'VIC'],
 ['NSW', 'SA', 'TAS', 'VIC'],
 ['NSW', 'QLD', 'SA', 'TAS', 'VIC']]

for i in l:
    input_file_path = 'C:\\Users\\vm-user\\Downloads\\aemo_merged.csv'
    selected_columns = ['SETTLEMENTDATE']
    for j in i:
        selected_columns.append(f'TOTALDEMAND_{j}')
        selected_columns.append(f'RRP_{j}')
    output_file_path = f'C:\\Users\\vm-user\\Downloads\\{"_".join(i)}.csv'
    filter_columns_and_save(input_file_path, selected_columns, output_file_path)

Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\TAS.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\QLD.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\SA.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_QLD.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\QLD_SA.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\TAS_VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_SA_VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_QLD_VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_QLD_SA.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_TAS_VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_QLD_SA_VIC.csv
Filtered dataframe saved to C:\Users\vm-user\Downloads\NSW_SA_T

In [3]:
filter_columns_and_save(
    "C:\\Users\\vm-user\\Downloads\\aemo_merged.csv", 
    ["SETTLEMENTDATE", "TOTALDEMAND_NSW", "RRP_NSW", "TOTALDEMAND_QLD", "RRP_QLD", "TOTALDEMAND_VIC", "RRP_VIC"], 
    "C:\\Users\\vm-user\\Downloads\\aemo_nsw_qld_vic.csv"
)

Filtered dataframe saved to C:\Users\vm-user\Downloads\aemo_nsw_qld_vic.csv


In [4]:
filter_columns_and_save(
    "C:\\Users\\vm-user\\Downloads\\aemo_merged.csv", 
    ["SETTLEMENTDATE", "TOTALDEMAND_NSW", "RRP_NSW", "TOTALDEMAND_QLD", "RRP_QLD", "TOTALDEMAND_VIC", "RRP_VIC", "TOTALDEMAND_TAS", "RRP_TAS"], 
    "C:\\Users\\vm-user\\Downloads\\aemo_nsw_qld_vic_tas.csv"
)

Filtered dataframe saved to C:\Users\vm-user\Downloads\aemo_nsw_qld_vic_tas.csv


# EU

In [115]:
all_countries = pd.read_csv("C:\\Users\\vm-user\\Downloads\\all_countries.csv")

In [116]:
all_countries.head()

,Country,ISO3 Code,Datetime (UTC),Datetime (Local),Price (EUR/MWhe)
0,Austria,AUT,2015-01-01 00:00:00,2015-01-01 01:00:00,22.34
1,Belgium,BEL,2015-01-01 00:00:00,2015-01-01 01:00:00,36.56
2,Czechia,CZE,2015-01-01 00:00:00,2015-01-01 01:00:00,24.20
3,Denmark,DNK,2015-01-01 00:00:00,2015-01-01 01:00:00,18.29
4,Estonia,EST,2015-01-01 00:00:00,2015-01-01 02:00:00,23.37


In [117]:
countries_map = {
    'Austria': 'AT',
    'Belgium': 'BE',
    'Bulgaria': 'BG',
    'Croatia': 'HR',
    'Czechia': 'CZ',
    'Denmark': 'DK',
    'Estonia': 'EE',
    'Finland': 'FI',
    'France': 'FR',
    'Germany': 'DE',
    'Greece': 'GR',
    'Hungary': 'HU',
    'Ireland': 'IE',
    'Italy': 'IT',
    'Latvia': 'LV',
    'Lithuania': 'LT',
    'Luxembourg': 'LU',
    'Montenegro': 'ME',
    'Netherlands': 'NL',
    'North Macedonia': 'MK',
    'Norway': 'NO',
    'Poland': 'PL',
    'Portugal': 'PT',
    'Romania': 'RO',
    'Serbia': 'RS',
    'Slovakia': 'SK',
    'Slovenia': 'SI',
    'Spain': 'ES',
    'Sweden': 'SE',
    'Switzerland': 'CH',
    'United Kingdom': 'GB'
}

In [118]:
len(set(countries_map.values())), len(countries_map.values())

(31, 31)

In [119]:
# Replace the Country column values with their two-letter codes
all_countries['Country'] = all_countries['Country'].map(countries_map)

# Optionally, check the result
print("Unique two-letter country codes in Country column:", all_countries['Country'].unique())

Unique two-letter country codes in Country column: ['AT' 'BE' 'CZ' 'DK' 'EE' 'FI' 'FR' 'DE' 'GR' 'HU' 'IT' 'LV' 'LT' 'LU'
 'NL' 'NO' 'PL' 'PT' 'RO' 'SK' 'SI' 'ES' 'SE' 'CH' 'GB' 'BG' 'RS' 'HR'
 'ME' 'MK' 'IE']


In [120]:
len(set(countries_map.values())), len(countries_map.values())

(31, 31)

In [121]:
import pandas as pd

def pivot_and_resample(df):
    """
    Transforms the dataset by pivoting countries into columns and resampling the time series.

    Parameters:
    df (pd.DataFrame): Input dataframe containing country-wise electricity prices.

    Returns:
    pd.DataFrame: Resampled dataframe with hourly prices for each country as separate columns.
    """
    # Convert 'Datetime (UTC)' to datetime format
    df['Datetime (UTC)'] = pd.to_datetime(df['Datetime (UTC)'])

    # Drop 'Datetime (Local)' as it's not needed
    df = df.drop(columns=['Datetime (Local)'])

    # Rename 'Price (EUR/MWhe)' to 'Price'
    df = df.rename(columns={'Price (EUR/MWhe)': 'Price'})

    # Pivot the table to make countries as columns
    df_pivoted = df.pivot_table(index='Datetime (UTC)', columns='Country', values='Price', aggfunc='mean')

    # Resample to ensure hourly intervals from min to max timestamp
    df_resampled = df_pivoted.resample('h').mean()

    # Reset index to bring datetime back as a column
    df_resampled.reset_index(inplace=True)

    return df_resampled

# Example usage
final_df = pivot_and_resample(all_countries)

In [122]:
df = final_df.copy()
for col in df.columns[1:]:
    non_null_series = df[col].dropna()  # Remove NaN values to find the first non-null value
    if not non_null_series.empty:
        first_index = non_null_series.index[0]  # Find the first non-null value's index
        first_value = non_null_series.iloc[0]  # Get the first non-null value
        
        # Replace consecutive occurrences with NaN, starting from first_index
        for i in range(first_index + 1, len(df)):
            if pd.isna(df[col].iloc[i]) or df[col].iloc[i] != first_value:
                break  # Stop if a different value or NaN appears
            df.loc[i-1, col] = float("nan")  # Set previous consecutive occurrences to NaN

# Remove initial rows where all country columns are NaN (excluding 'Datetime')
df_cleaned = df.dropna(subset=df.columns[1:], how='all').reset_index(drop=True)

In [123]:
def has_1hr_intervals(time_series):
    time_series = pd.to_datetime(time_series).sort_values()
    expected_interval = pd.Timedelta(hours=1)
    return (time_series.diff().dropna() == expected_interval).all()

In [124]:
has_1hr_intervals(df_cleaned['Datetime (UTC)'])

True

In [125]:
df_cleaned

Country,Datetime (UTC),AT,BE,BG,CH,CZ,DE,DK,EE,ES,...,MK,NL,NO,PL,PT,RO,RS,SE,SI,SK
0,2015-01-01 00:00:00,NaN,NaN,NaN,43.43,24.20,NaN,18.29,23.37,48.10,...,NaN,NaN,27.35,NaN,48.10,NaN,NaN,23.37,23.25,24.20
1,2015-01-01 01:00:00,NaN,NaN,NaN,38.08,22.06,NaN,16.04,19.33,47.33,...,NaN,NaN,27.23,NaN,47.33,NaN,NaN,19.33,22.20,22.06
2,2015-01-01 02:00:00,NaN,NaN,NaN,35.47,20.27,NaN,14.60,17.66,42.27,...,NaN,NaN,27.15,NaN,42.27,NaN,NaN,17.66,19.56,20.27
3,2015-01-01 03:00:00,NaN,NaN,NaN,30.83,19.17,NaN,14.95,17.53,38.41,...,NaN,NaN,27.14,NaN,38.41,NaN,NaN,17.53,18.88,19.17
4,2015-01-01 04:00:00,NaN,NaN,NaN,28.26,17.90,NaN,14.50,18.07,35.72,...,NaN,NaN,27.29,NaN,35.72,NaN,NaN,18.07,18.39,17.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88818,2025-02-17 18:00:00,214.41,215.46,224.35,203.38,216.50,214.10,215.38,219.35,215.52,...,148.2,215.43,94.09,218.20,215.52,216.86,223.78,157.34,216.46,217.09
88819,2025-02-17 19:00:00,178.47,177.95,224.35,171.35,179.08,181.78,177.87,180.91,200.00,...,148.2,177.93,88.03,180.90,200.00,179.47,195.21,130.24,179.03,179.71
88820,2025-02-17 20:00:00,159.23,155.38,224.35,159.98,151.40,161.50,155.19,155.95,155.62,...,148.2,155.30,82.59,154.09,155.62,165.32,174.31,110.05,161.59,172.18
88821,2025-02-17 21:00:00,159.23,139.51,224.35,159.98,151.40,161.50,155.19,155.95,140.00,...,148.2,139.39,82.59,154.09,155.62,161.88,174.31,110.05,161.59,172.18


In [126]:
import pandas as pd

def filter_and_clean_dataframe(df, selected_columns):
    """
    Filters the given DataFrame to retain only the selected columns, 
    and removes initial and bottom-most rows where all selected columns are NaN.
    
    Parameters:
        df (pd.DataFrame): The original DataFrame.
        selected_columns (list): List of column names to retain.
    
    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    # Ensure 'Datetime (UTC)' is included in the filtered columns
    filtered_columns = ['Datetime (UTC)'] + selected_columns if 'Datetime (UTC)' in df.columns else selected_columns
    df_filtered = df[filtered_columns]

    # Drop initial rows where all selected columns (excluding 'Datetime') are NaN
    start_index = df_filtered[selected_columns].first_valid_index()
    df_cleaned = df_filtered.loc[start_index:].reset_index(drop=True)
    
    # Drop bottom-most rows where all selected columns (excluding 'Datetime') are NaN
    end_index = df_cleaned[selected_columns].last_valid_index()
    df_cleaned = df_cleaned.loc[:end_index].reset_index(drop=True)
    df_cleaned.fillna(10**6, inplace=True)  # Replace NaNs with a large number

    return df_cleaned

In [127]:
from geopy.geocoders import Nominatim
import requests
import numpy as np

# List of countries
countries = ['Austria', 'Belgium', 'Czechia', 'Denmark', 'Estonia', 'Finland',
'France', 'Germany', 'Greece', 'Hungary', 'Italy', 'Latvia',
'Lithuania', 'Luxembourg', 'Netherlands', 'Norway', 'Poland',
'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
'Switzerland', 'United Kingdom', 'Bulgaria', 'Serbia', 'Croatia',
'Montenegro', 'North Macedonia', 'Ireland']

lat = [47.516231, 50.503887, 49.817492, 56.26392, 58.595272, 61.92411,
       46.227638, 51.165691, 39.074208, 47.162494, 41.87194, 56.879635,
       55.169438, 49.815273, 52.132633, 60.472024, 51.919438, 39.399872,
       45.943161, 48.669026, 46.151241, 40.463667, 60.128161, 46.818188,
       55.378051, 42.733883, 44.016521, 45.1, 42.708678, 41.608635, 53.41291]

long = [14.550072, 4.469936, 15.472962, 9.501785, 25.013607, 25.748151,
        2.213749, 10.451526, 21.824312, 19.503304, 12.56738, 24.603189,
        23.881275, 6.129583, 5.291266, 8.468946, 19.145136, -8.224454,
        24.96676, 19.699024, 14.995463, -3.74922, 18.643501, 8.227512,
        -3.435973, 25.48583, 21.005859, 15.2, 19.37439, 21.745275, -8.24389]

area_km_sq = [82520, 30280, 77187, 42434, 42388, 303815, 
       640427, 349390, 128900, 91260, 295717, 62230,
       62610, 2574, 33893, 366704, 304255,
       91606, 230080, 48080, 20151, 498980, 407284,
       39510, 242741, 108489, 88499, 55974,
       13452, 25220, 68883]

eu_data = {
    'countries': countries,
    'lat': lat,
    'long': long,
    'area_km_sq': area_km_sq
}


def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Function to calculate the effective radius based on area
def effective_radius(area):
    return np.sqrt(area / np.pi)

# Prepare the data into a DataFrame
eu_df = pd.DataFrame(eu_data)

# Calculate the effective radius for each country
eu_df['effective_radius'] = eu_df['area_km_sq'].apply(effective_radius)

# Initialize list to store both normalized and physical distances
distances = []

# Loop through pairs of countries to calculate distances
for i in range(len(eu_df)):
    for j in range(len(eu_df)):
        # Get the coordinates, areas and radii
        lat1, lon1, r1 = eu_df.iloc[i]['lat'], eu_df.iloc[i]['long'], eu_df.iloc[i]['effective_radius']
        lat2, lon2, r2 = eu_df.iloc[j]['lat'], eu_df.iloc[j]['long'], eu_df.iloc[j]['effective_radius']
        
        # Calculate the physical distance between the centroids using the Haversine formula
        distance = haversine(lat1, lon1, lat2, lon2)
        
        # Calculate the normalized distance
        D_norm = distance / (r1 + r2)
        
        # Append the results to the list
        distances.append({
            'Country 1': eu_df.iloc[i]['countries'],
            'Country 2': eu_df.iloc[j]['countries'],
            'Physical Distance (km)': distance,
            'Normalized Distance': D_norm
        })

distances_df = pd.DataFrame(distances)

In [128]:
distances_df

,Country 1,Country 2,Physical Distance (km),Normalized Distance
0,Austria,Austria,0.000000,0.000000
1,Austria,Belgium,805.904977,3.096702
2,Austria,Czechia,264.705969,0.830276
3,Austria,Denmark,1031.865905,3.707866
4,Austria,Estonia,1412.866158,5.078086
...,...,...,...,...
956,Ireland,Serbia,2364.256758,7.483855
957,Ireland,Croatia,1923.646715,6.832214
958,Ireland,Montenegro,2350.806279,11.010233
959,Ireland,North Macedonia,2578.886347,10.850582


In [3]:
import pandas as pd
import math

def closest_locations(df, location_col, other_location_col, distance_col, n):
    # Create an empty dictionary to store the results
    result = {}
    
    # Get unique values in the location column
    unique_locations = df[location_col].unique()
    
    for location in unique_locations:
        # Filter the dataframe for rows corresponding to the current location
        location_df = df[df[location_col] == location]
        
        # Sort the rows based on the distance column
        location_df = location_df.sort_values(by=distance_col)
        
        # Create subsets that progressively add the next 'n' closest locations
        closest_n_locations = []
            
        num_steps = math.ceil(len(location_df) / n)
        for i in range(1, num_steps + 1):
            # Ensure we don't go past the length of the DataFrame
            end_index = min(i * n, len(location_df))
            closest_n_locations.append(location_df.iloc[:end_index][other_location_col].tolist())        # Add the result for the current location to the dictionary
            result[location] = closest_n_locations
    
    return result

In [4]:
len(distances_df[distances_df['Country 1'] == 'Austria'].sort_values(by='Normalized Distance'))

KeyError: 'Country 1'

In [4]:
def unique_subsets(closest_dict):
    # Set to hold unique subsets (as sorted tuples)
    unique_sets = set()

    # Iterate over each location's list of subsets
    for location, subsets in closest_dict.items():
        for subset in subsets:
            # Convert each subset to a sorted tuple (for order-insensitivity)
            unique_sets.add(tuple(sorted(subset)))
    
    # Convert the set of unique tuples back to a list of lists
    unique_subsets_list = [list(subset) for subset in unique_sets]
    
    # Sort the subsets by length
    unique_subsets_list.sort(key=len)
    
    return unique_subsets_list

In [132]:
country_subsets = unique_subsets(closest_locations(distances_df, 'Country 1', 'Country 2', 'Normalized Distance', 10))

In [133]:
len(country_subsets)

58

In [134]:
df_cleaned['Datetime (UTC)'] = pd.to_datetime(df_cleaned['Datetime (UTC)'])
df_cleaned['Datetime (UTC)'] = df_cleaned['Datetime (UTC)'].dt.strftime('%Y-%m-%d %H:%M')

for country in countries_map:
    df_to_save = filter_and_clean_dataframe(df_cleaned, [countries_map[country]])
    output_file_path = f"C:\\Users\\vm-user\\Downloads\\eu_{countries_map[country]}.csv"
    # Print a message to see if the df has null values
    if df_to_save.isnull().values.any():
        print(f"Warning: DataFrame for {countries_map[country]} contains null values.")
    df_to_save.to_csv(output_file_path, index=False)
    print(f"Saved: {output_file_path}")

for subset in country_subsets:
    subset = [countries_map[country] for country in subset]
    df_to_save = filter_and_clean_dataframe(df_cleaned, subset)
    if df_to_save.isnull().values.any():
        print(f"Warning: DataFrame for {countries_map[country]} contains null values.")
    output_file_path = f"C:\\Users\\vm-user\\Downloads\\eu_{'_'.join(subset)}.csv"
    df_to_save.to_csv(output_file_path, index=False)
    print(f"Saved: {output_file_path}")

Saved: C:\Users\vm-user\Downloads\eu_AT.csv
Saved: C:\Users\vm-user\Downloads\eu_BE.csv
Saved: C:\Users\vm-user\Downloads\eu_BG.csv
Saved: C:\Users\vm-user\Downloads\eu_HR.csv
Saved: C:\Users\vm-user\Downloads\eu_CZ.csv
Saved: C:\Users\vm-user\Downloads\eu_DK.csv
Saved: C:\Users\vm-user\Downloads\eu_EE.csv
Saved: C:\Users\vm-user\Downloads\eu_FI.csv
Saved: C:\Users\vm-user\Downloads\eu_FR.csv
Saved: C:\Users\vm-user\Downloads\eu_DE.csv
Saved: C:\Users\vm-user\Downloads\eu_GR.csv
Saved: C:\Users\vm-user\Downloads\eu_HU.csv
Saved: C:\Users\vm-user\Downloads\eu_IE.csv
Saved: C:\Users\vm-user\Downloads\eu_IT.csv
Saved: C:\Users\vm-user\Downloads\eu_LV.csv
Saved: C:\Users\vm-user\Downloads\eu_LT.csv
Saved: C:\Users\vm-user\Downloads\eu_LU.csv
Saved: C:\Users\vm-user\Downloads\eu_ME.csv
Saved: C:\Users\vm-user\Downloads\eu_NL.csv
Saved: C:\Users\vm-user\Downloads\eu_MK.csv
Saved: C:\Users\vm-user\Downloads\eu_NO.csv
Saved: C:\Users\vm-user\Downloads\eu_PL.csv
Saved: C:\Users\vm-user\Download

In [ ]:
df_cleaned.isna().sum()

In [59]:
len(set(country_subsets[-1]) & set(country_subsets[-2])), len(set(country_subsets[-1]) | set(country_subsets[-2]))

(29, 31)

In [60]:
len(set(country_subsets[-1]))

30

# NYISO

In [153]:
nyiso_data_df = pd.read_csv("C:\\Users\\vm-user\\Downloads\\nyiso.csv")

In [2]:
from geopy.geocoders import Nominatim
import requests
import pandas as pd
import numpy as np

# List of countries
regions = ['CAPITL', 'CENTRL', 'MHK VL', 'HUD VL', 'LONGIL', 'N.Y.C.', 'WEST', 'GENESE', 'NORTH', 'DUNWOD', 'MILLWD']

long = [-73.9820406, -76.7300675, -75.1443401, -74.0753086, -73.0188755, -73.9247654, -78.6001935, -77.5107558, -74.0169224, -73.7846149, -73.8141997]

lat = [43.1823555, 42.6094099, 43.4161459, 41.7112736, 40.8443418, 40.6944843, 42.6613719, 42.8694038, 44.5393904, 41.0350397, 41.2394963]

area_km_sq = [41206.36, 39600.998, 74038.96, 18453.5, 5795.6, 1566.34, 29527.06, 9535.63, 15199.63, 905.66, 743.044]

nyiso_data = {
    'regions': regions,
    'lat': lat,
    'long': long,
    'area_km_sq': area_km_sq
}


def haversine(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Function to calculate the effective radius based on area
def effective_radius(area):
    return np.sqrt(area / np.pi)

# Prepare the data into a DataFrame
nyiso_df = pd.DataFrame(nyiso_data)

# Calculate the effective radius for each country
nyiso_df['effective_radius'] = nyiso_df['area_km_sq'].apply(effective_radius)

# Initialize list to store both normalized and physical distances
distances = []

# Loop through pairs of countries to calculate distances
for i in range(len(nyiso_df)):
    for j in range(len(nyiso_df)):
        # Get the coordinates, areas and radii
        lat1, lon1, r1 = nyiso_df.iloc[i]['lat'], nyiso_df.iloc[i]['long'], nyiso_df.iloc[i]['effective_radius']
        lat2, lon2, r2 = nyiso_df.iloc[j]['lat'], nyiso_df.iloc[j]['long'], nyiso_df.iloc[j]['effective_radius']
        
        # Calculate the physical distance between the centroids using the Haversine formula
        distance = haversine(lat1, lon1, lat2, lon2)
        
        # Calculate the normalized distance
        D_norm = distance / (r1 + r2)
        
        # Append the results to the list
        distances.append({
            'Region 1': nyiso_df.iloc[i]['regions'],
            'Region 2': nyiso_df.iloc[j]['regions'],
            'Physical Distance (km)': distance,
            'Normalized Distance': D_norm
        })

distances_df = pd.DataFrame(distances)

In [5]:
distances_df[distances_df['Region 1'] == 'GENESE'].sort_values(by='Normalized Distance')

,Region 1,Region 2,Physical Distance (km),Normalized Distance
84,GENESE,GENESE,0.000000,0.000000
78,GENESE,CENTRL,70.004366,0.418268
83,GENESE,WEST,91.892089,0.604392
79,GENESE,MHK VL,201.382602,0.965355
77,GENESE,CAPITL,288.925037,1.703364
80,GENESE,HUD VL,310.512718,2.357101
85,GENESE,NORTH,336.639628,2.700667
81,GENESE,LONGIL,434.763550,4.434348
82,GENESE,N.Y.C.,383.206844,4.949563
87,GENESE,MILLWD,354.905062,5.036074


In [29]:
distances_df[distances_df['Region 1'] == 'CAPITL'].sort_values(by='Normalized Distance')

,Region 1,Region 2,Physical Distance (km),Normalized Distance
0,CAPITL,CAPITL,0.000000,0.000000
2,CAPITL,MHK VL,97.585199,0.364065
8,CAPITL,NORTH,150.921305,0.819851
3,CAPITL,HUD VL,163.755724,0.856605
1,CAPITL,CENTRL,232.731997,1.026153
10,CAPITL,MILLWD,216.477736,1.666419
7,CAPITL,GENESE,288.925037,1.703364
4,CAPITL,LONGIL,271.874938,1.726432
6,CAPITL,WEST,380.423305,1.798913
9,CAPITL,DUNWOD,239.325181,1.819885


In [45]:
import itertools

def average_distance_between_regions(regions: list, distances_df: pd.DataFrame, column = "Normalized Distance") -> float:
    """
    Given a list of region names and a dataframe with distance information,
    this function computes the average distance between all unique pairs.

    distances_df should have columns "Region 1", "Region 2", and "Normalized Distance".
    """
    # Generate all unique combinations (pairs) of regions
    if len(regions) == 1:
        return 0.0
    pairs = list(itertools.combinations(regions, 2))
    pairs = [pair for pair in pairs if (pair[0] == regions[0] or pair[1] == regions[0])]
    distances = []
    for r1, r2 in pairs:
        print(r1, r2)
        # Filter for rows where the pair (in any order) matches r1 and r2.
        mask = (((distances_df["Region 1"] == r1) & (distances_df["Region 2"] == r2)) |
                ((distances_df["Region 1"] == r2) & (distances_df["Region 2"] == r1)))
        
        row = distances_df.loc[mask]
        if not row.empty:
            # Assumes one row per unique pair.
            distance = row.iloc[0][column]
            distances.append(distance)
    
    # Check if any matches were found; if not, return None (or you can raise an error)
    if distances:
        return sum(distances) / len(distances)
    else:
        return None

In [46]:
subset_captil = ['CAPITL', 'CAPITL_MHK VL_NORTH', 'CAPITL_CENTRL_HUD VL_MHK VL_MILLWD_NORTH', 'CAPITL_CENTRL_GENESE_HUD VL_LONGIL_MHK VL_MILLWD_NORTH_WEST', 'CAPITL_CENTRL_DUNWOD_GENESE_HUD VL_LONGIL_MHK VL_MILLWD_N.Y.C._NORTH_WEST']

for subset in subset_captil:
    # Split the string at "_" to infer individual regions
    regions = subset.split("_")
    avg_dist = average_distance_between_regions(regions, distances_df)
    print(f"Regions: {regions}\nAverage distance: {avg_dist}\n")

Regions: ['CAPITL']
Average distance: 0.0

CAPITL MHK VL
CAPITL NORTH
Regions: ['CAPITL', 'MHK VL', 'NORTH']
Average distance: 0.5919577291673273

CAPITL CENTRL
CAPITL HUD VL
CAPITL MHK VL
CAPITL MILLWD
CAPITL NORTH
Regions: ['CAPITL', 'CENTRL', 'HUD VL', 'MHK VL', 'MILLWD', 'NORTH']
Average distance: 0.9466183041464564

CAPITL CENTRL
CAPITL GENESE
CAPITL HUD VL
CAPITL LONGIL
CAPITL MHK VL
CAPITL MILLWD
CAPITL NORTH
CAPITL WEST
Regions: ['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'NORTH', 'WEST']
Average distance: 1.2452250995593415

CAPITL CENTRL
CAPITL DUNWOD
CAPITL GENESE
CAPITL HUD VL
CAPITL LONGIL
CAPITL MHK VL
CAPITL MILLWD
CAPITL N.Y.C.
CAPITL NORTH
CAPITL WEST
Regions: ['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.', 'NORTH', 'WEST']
Average distance: 1.380337081798436



In [ ]:
subset_captil = ['CAPITL', 'CAPITL_MHK VL_NORTH', 'CAPITL_CENTRL_HUD VL_MHK VL_MILLWD_NORTH', 'CAPITL_CENTRL_GENESE_HUD VL_LONGIL_MHK VL_MILLWD_NORTH_WEST', 'CAPITL_CENTRL_DUNWOD_GENESE_HUD VL_LONGIL_MHK VL_MILLWD_N.Y.C._NORTH_WEST']

for subset in subset_captil:
    print(f"Average distance: {average_distance_between_regions(subset.split('_'), distances_df, 'Physical Distance (km)')}")

Average distance: 134.19151078096544
Average distance: 214.51058674685788
Average distance: 263.2697326129474
Average distance: 256.95182056406793


In [27]:
d = closest_locations(distances_df, 'Region 1', 'Region 2', 'Normalized Distance', 3)

print('[', end='')
for l in d['WEST']:
    l.sort()
    print("'", end='')
    print('_'.join(l), end='')
    print("',", end='')
print(']', end='')

['CENTRL_GENESE_WEST','CAPITL_CENTRL_GENESE_HUD VL_MHK VL_WEST','CAPITL_CENTRL_GENESE_HUD VL_LONGIL_MHK VL_N.Y.C._NORTH_WEST','CAPITL_CENTRL_DUNWOD_GENESE_HUD VL_LONGIL_MHK VL_MILLWD_N.Y.C._NORTH_WEST',]

In [7]:
closest_locations(distances_df, 'Region 1', 'Region 2', 'Normalized Distance', 3)

{'CAPITL': [['CAPITL', 'MHK VL', 'NORTH'],
  ['CAPITL', 'MHK VL', 'NORTH', 'HUD VL', 'CENTRL', 'MILLWD'],
  ['CAPITL',
   'MHK VL',
   'NORTH',
   'HUD VL',
   'CENTRL',
   'MILLWD',
   'GENESE',
   'LONGIL',
   'WEST'],
  ['CAPITL',
   'MHK VL',
   'NORTH',
   'HUD VL',
   'CENTRL',
   'MILLWD',
   'GENESE',
   'LONGIL',
   'WEST',
   'DUNWOD',
   'N.Y.C.']],
 'CENTRL': [['CENTRL', 'GENESE', 'MHK VL'],
  ['CENTRL', 'GENESE', 'MHK VL', 'WEST', 'CAPITL', 'HUD VL'],
  ['CENTRL',
   'GENESE',
   'MHK VL',
   'WEST',
   'CAPITL',
   'HUD VL',
   'NORTH',
   'MILLWD',
   'DUNWOD'],
  ['CENTRL',
   'GENESE',
   'MHK VL',
   'WEST',
   'CAPITL',
   'HUD VL',
   'NORTH',
   'MILLWD',
   'DUNWOD',
   'N.Y.C.',
   'LONGIL']],
 'MHK VL': [['MHK VL', 'CAPITL', 'CENTRL'],
  ['MHK VL', 'CAPITL', 'CENTRL', 'NORTH', 'HUD VL', 'GENESE'],
  ['MHK VL',
   'CAPITL',
   'CENTRL',
   'NORTH',
   'HUD VL',
   'GENESE',
   'WEST',
   'MILLWD',
   'DUNWOD'],
  ['MHK VL',
   'CAPITL',
   'CENTRL',
   'NORTH',
 

In [6]:
len(unique_subsets(closest_locations(distances_df, 'Region 1', 'Region 2', 'Normalized Distance', 3)))

19

In [8]:
region_subsets = unique_subsets(closest_locations(distances_df, 'Region 1', 'Region 2', 'Normalized Distance', 3))

In [9]:
len(region_subsets)

19

In [10]:
region_subsets

[['CAPITL', 'CENTRL', 'MHK VL'],
 ['CENTRL', 'GENESE', 'WEST'],
 ['DUNWOD', 'HUD VL', 'LONGIL'],
 ['CENTRL', 'GENESE', 'MHK VL'],
 ['CAPITL', 'MHK VL', 'NORTH'],
 ['DUNWOD', 'HUD VL', 'MILLWD'],
 ['DUNWOD', 'HUD VL', 'N.Y.C.'],
 ['DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.'],
 ['CAPITL', 'DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD'],
 ['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'MHK VL', 'WEST'],
 ['CAPITL', 'CENTRL', 'HUD VL', 'MHK VL', 'MILLWD', 'NORTH'],
 ['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'MHK VL', 'NORTH'],
 ['CAPITL', 'CENTRL', 'HUD VL', 'MHK VL', 'NORTH', 'WEST'],
 ['CAPITL',
  'CENTRL',
  'DUNWOD',
  'HUD VL',
  'LONGIL',
  'MHK VL',
  'MILLWD',
  'N.Y.C.',
  'WEST'],
 ['CAPITL',
  'CENTRL',
  'DUNWOD',
  'HUD VL',
  'LONGIL',
  'MHK VL',
  'MILLWD',
  'N.Y.C.',
  'NORTH'],
 ['CAPITL',
  'CENTRL',
  'GENESE',
  'HUD VL',
  'LONGIL',
  'MHK VL',
  'MILLWD',
  'NORTH',
  'WEST'],
 ['CAPITL',
  'CENTRL',
  'GENESE',
  'HUD VL',
  'LONGIL',
  'MHK VL',
  'N.Y.C

In [14]:
region_subsets = sorted(region_subsets, key=len)
for i in region_subsets:
    print(i)

['CAPITL', 'CENTRL', 'MHK VL']
['DUNWOD', 'HUD VL', 'N.Y.C.']
['DUNWOD', 'HUD VL', 'MILLWD']
['CENTRL', 'GENESE', 'MHK VL']
['CAPITL', 'MHK VL', 'NORTH']
['DUNWOD', 'HUD VL', 'LONGIL']
['CENTRL', 'GENESE', 'WEST']
['CAPITL', 'CENTRL', 'HUD VL', 'MHK VL', 'MILLWD', 'NORTH']
['DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.']
['CAPITL', 'DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD']
['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'MHK VL', 'NORTH']
['CAPITL', 'CENTRL', 'HUD VL', 'MHK VL', 'NORTH', 'WEST']
['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'MHK VL', 'WEST']
['CAPITL', 'CENTRL', 'DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.', 'NORTH']
['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'MHK VL', 'MILLWD', 'NORTH', 'WEST']
['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'LONGIL', 'MHK VL', 'N.Y.C.', 'NORTH', 'WEST']
['CAPITL', 'CENTRL', 'DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.', 'WEST']
['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', '

In [20]:
!python "S:\\spatiotemporal-analysis\\spacetimeformer-main\\spacetimeformer\\train.py" spacetimeformer "aemo-NSW-QLD-SA-VIC" --epochs 100 --context_points 360 --target_points 48 --run_name aemo_NSW_QLD_SA_VIC --use_revin --use_seasonal_decomp --l2_coeff 0.0001 --batch_size 32 --base_lr 0.0001 --learning_rate 0.0001 --d_model 100 --d_ff 400 --enc_layers 1 --dec_layers 1 --dropout_emb .1 --dropout_ff .3 --d_qk 30 --d_v 30 --n_heads 10 --patience 10 --data_path "C:\\Users\\vm-user\\Downloads\\All Datasets\\All Datasets\\aemo\\NSW_QLD_SA_VIC.csv" --test_only --ckpt_path "C:\\Users\\vm-user\\Downloads\\aemo_NSW_QLD_SA_VICepoch=07.ckpt"

^C


In [ ]:
l = [
]

['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.', 'NORTH', 'WEST']

In [ ]:
# Correct

l = [['NSW'],
 ['TAS'],
 ['QLD'],
 ['VIC'],
 ['SA'],
 ['NSW', 'QLD'],
 ['NSW', 'VIC'],
 ['QLD', 'SA'],
 ['TAS', 'VIC'],
 ['NSW', 'SA', 'VIC'],
 ['NSW', 'QLD', 'VIC'],
 ['NSW', 'QLD', 'SA'],
 ['NSW', 'TAS', 'VIC'],
 ['NSW', 'QLD', 'SA', 'VIC'],
 ['NSW', 'SA', 'TAS', 'VIC'],
 ['NSW', 'QLD', 'SA', 'TAS', 'VIC']]


CAPITL
CENTRL
MHK VL
HUD VL
LONGIL
N.Y.C.
WEST
GENESE
NORTH
DUNWOD
MILLWD
['CAPITL', 'CENTRL', 'MHK VL']
['DUNWOD', 'HUD VL', 'N.Y.C.']
['DUNWOD', 'HUD VL', 'MILLWD']
['CENTRL', 'GENESE', 'MHK VL']
['CAPITL', 'MHK VL', 'NORTH']
['DUNWOD', 'HUD VL', 'LONGIL']
['CENTRL', 'GENESE', 'WEST']
['CAPITL', 'CENTRL', 'HUD VL', 'MHK VL', 'MILLWD', 'NORTH']
['DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.']
['CAPITL', 'DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD']
['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'MHK VL', 'NORTH']
['CAPITL', 'CENTRL', 'HUD VL', 'MHK VL', 'NORTH', 'WEST']
['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'MHK VL', 'WEST']
['CAPITL', 'CENTRL', 'DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.', 'NORTH']
['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'MHK VL', 'MILLWD', 'NORTH', 'WEST']
['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'LONGIL', 'MHK VL', 'N.Y.C.', 'NORTH', 'WEST']
['CAPITL', 'CENTRL', 'DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.', 'WEST']
['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'NORTH', 'WEST']
['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.', 'NORTH', 'WEST']

In [ ]:
for region in regions:
    print(region)

CAPITL
CENTRL
MHK VL
HUD VL
LONGIL
N.Y.C.
WEST
GENESE
NORTH
DUNWOD
MILLWD


In [160]:
nyiso_data_df.isna().sum()

Time Stamp    0
CAPITL        0
CENTRL        0
DUNWOD        0
GENESE        0
HUD VL        0
LONGIL        0
MHK VL        0
MILLWD        0
N.Y.C.        0
NORTH         0
WEST          0
dtype: int64

In [161]:
import pandas as pd

def filter_nyiso(df, selected_columns):
    """
    Filters the given DataFrame to retain only the selected columns, 
    and removes initial and bottom-most rows where all selected columns are NaN.
    
    Parameters:
        df (pd.DataFrame): The original DataFrame.
        selected_columns (list): List of column names to retain.
    
    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    # Ensure 'Datetime (UTC)' is included in the filtered columns
    filtered_columns = ['Time Stamp'] + selected_columns if 'Time Stamp' in df.columns else selected_columns
    df_filtered = df[filtered_columns]
    return df_filtered

In [162]:
nyiso_data_df['Time Stamp'] = pd.to_datetime(nyiso_data_df['Time Stamp'])
nyiso_data_df['Time Stamp'] = nyiso_data_df['Time Stamp'].dt.strftime('%Y-%m-%d %H:%M')

for region in regions:
    df_to_save = filter_nyiso(nyiso_data_df, [region])
    output_file_path = f"C:\\Users\\vm-user\\Downloads\\nyiso_{region}.csv"
    # Print a message to see if the df has null values
    if df_to_save.isnull().values.any():
        print(f"Warning: DataFrame for {region} contains null values.")

    df_to_save.to_csv(output_file_path, index=False)
    print(f"Saved: {output_file_path}")

for subset in region_subsets:
    df_to_save = filter_nyiso(nyiso_data_df, subset)
    output_file_path = f"C:\\Users\\vm-user\\Downloads\\nyiso_{'_'.join(subset)}.csv"
    # Print a message to see if the df has null values
    if df_to_save.isnull().values.any():
        print(f"Warning: DataFrame for {subset} contains null values.")
    df_to_save.to_csv(output_file_path, index=False)
    print(f"Saved: {output_file_path}")

Saved: C:\Users\vm-user\Downloads\nyiso_CAPITL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_CENTRL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_MHK VL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_HUD VL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_LONGIL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_N.Y.C..csv
Saved: C:\Users\vm-user\Downloads\nyiso_WEST.csv
Saved: C:\Users\vm-user\Downloads\nyiso_GENESE.csv
Saved: C:\Users\vm-user\Downloads\nyiso_NORTH.csv
Saved: C:\Users\vm-user\Downloads\nyiso_DUNWOD.csv
Saved: C:\Users\vm-user\Downloads\nyiso_MILLWD.csv
Saved: C:\Users\vm-user\Downloads\nyiso_CENTRL_GENESE_WEST.csv
Saved: C:\Users\vm-user\Downloads\nyiso_DUNWOD_HUD VL_N.Y.C..csv
Saved: C:\Users\vm-user\Downloads\nyiso_DUNWOD_HUD VL_LONGIL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_DUNWOD_HUD VL_MILLWD.csv
Saved: C:\Users\vm-user\Downloads\nyiso_CAPITL_MHK VL_NORTH.csv
Saved: C:\Users\vm-user\Downloads\nyiso_CENTRL_GENESE_MHK VL.csv
Saved: C:\Users\vm-user\Downloads\nyiso_CAPITL_CENTRL_M

In [ ]:
# ALL

nsw_qld_sa_vic_tas = {
    "test/acc": 0.9999704957008362,
    "test/class_loss": 0.0018233139999210835,
    "test/forecast_loss": 2.3405966758728027,
    "test/loss": 2.340778112411499,
    "test/mae": 234.49581909179688,
    "test/mape": 3.645725727081299,
    "test/mse": 232049.1875,
    "test/norm_mae": 0.5295851230621338,
    "test/norm_mse": 2.3405728340148926,
    "test/recon_loss": -1.0,
    "test/smape": 0.47333967685699463
}

nsw = {
    "test/acc": 0.9999672174453735,
    "test/class_loss": 0.0005202997126616538,
    "test/forecast_loss": 3.6325647830963135,
    "test/loss": 3.6326160430908203,
    "test/mae": 396.374755859375,
    "test/mape": 1.3492451906204224,
    "test/mse": 530329.0,
    "test/norm_mae": 0.5416648387908936,
    "test/norm_mse": 3.632528066635132,
    "test/recon_loss": -1.0,
    "test/smape": 0.3519725799560547
}

nsw_qld = {
    "test/acc": 0.99991375207901,
    "test/class_loss": 0.0015330464811995625,
    "test/forecast_loss": 2.8653838634490967,
    "test/loss": 2.865537643432617,
    "test/mae": 311.83941650390625,
    "test/mape": 1.163816213607788,
    "test/mse": 357305.09375,
    "test/norm_mae": 0.48017483949661255,
    "test/norm_mse": 2.86535382270813,
    "test/recon_loss": -1.0,
    "test/smape": 0.36608725786209106
}

# additional
nsw_qld_vic_tas = {
    "test/acc": 0.9996684789657593,
    "test/class_loss": 0.0025208292063325644,
    "test/forecast_loss": 2.498936176300049,
    "test/loss": 2.4991865158081055,
    "test/mae": 260.812744140625,
    "test/mape": 3.8596131801605225,
    "test/mse": 269812.625,
    "test/norm_mae": 0.5280795097351074,
    "test/norm_mse": 2.498910903930664,
    "test/recon_loss": -1.0,
    "test/smape": 0.42840543389320374
}

qld = {
    "test/acc": 0.9999993443489075,
    "test/class_loss": 4.931714283884503e-05,
    "test/forecast_loss": 2.4047129154205322,
    "test/loss": 2.4047179222106934,
    "test/mae": 308.598876953125,
    "test/mape": 1.373152494430542,
    "test/mse": 291895.40625,
    "test/norm_mae": 0.5714372992515564,
    "test/norm_mse": 2.4046895503997803,
    "test/recon_loss": -1.0,
    "test/smape": 0.41102471947669983
}

nsw_vic = {
    "test/acc": 1.0,
    "test/class_loss": 0.00022880075266584754,
    "test/forecast_loss": 2.2792913913726807,
    "test/loss": 2.279313564300537,
    "test/mae": 345.0459899902344,
    "test/mape": 4.416192054748535,
    "test/mse": 391612.71875,
    "test/norm_mae": 0.5041431188583374,
    "test/norm_mse": 2.279268980026245,
    "test/recon_loss": -1.0,
    "test/smape": 0.45057564973831177
}

nsw_qld_vic = {
    "test/acc": 0.9987006187438965,
    "test/class_loss": 0.014215604402124882,
    "test/forecast_loss": 2.2719411849975586,
    "test/loss": 2.2733612060546875,
    "test/mae": 308.4784851074219,
    "test/mape": 2.0575342178344727,
    "test/mse": 326476.125,
    "test/norm_mae": 0.483934611082077,
    "test/norm_mse": 2.271919012069702,
    "test/recon_loss": -1.0,
    "test/smape": 0.45590466260910034
}


NSW_SA_TAS_VIC = {
    'test/acc': 0.999994695186615,
    'test/class_loss': 6.406679312931374e-05,
    'test/forecast_loss': 2.291043281555176,
    'test/loss': 2.2910513877868652,
    'test/mae': 226.53167724609375,
    'test/mape': 3.820439100265503,
    'test/mse': 243321.28125,
    'test/norm_mae': 0.5128385424613953,
    'test/norm_mse': 2.291020393371582,
    'test/recon_loss': -1.0,
    'test/smape': 0.5012460947036743
}


NSW_QLD_SA_VIC = {
    'test/acc': 0.9999899864196777,
    'test/class_loss': 0.0009756253566592932,
    'test/forecast_loss': 2.066863775253296,
    'test/loss': 2.0669617652893066,
    'test/mae': 262.3127136230469,
    'test/mape': 2.3660895824432373,
    'test/mse': 264874.1875,
    'test/norm_mae': 0.4795970916748047,
    'test/norm_mse': 2.0668442249298096,
    'test/recon_loss': -1.0,
    'test/smape': 0.49104467034339905
}


NSW_TAS_VIC = {
    'test/acc': 0.9999978542327881,
    'test/class_loss': 0.00014460313832387328,
    'test/forecast_loss': 3.0250966548919678,
    'test/loss': 3.025111198425293,
    'test/mae': 286.9242248535156,
    'test/mape': 10.117528915405273,
    'test/mse': 332857.28125,
    'test/norm_mae': 0.6201017498970032,
    'test/norm_mse': 3.025066375732422,
    'test/recon_loss': -1.0,
    'test/smape': 0.4419724941253662
}


NSW_QLD_SA = {
    'test/acc': 1.0,
    'test/class_loss': 0.0001490989961894229,
    'test/forecast_loss': 3.5893325805664062,
    'test/loss': 3.5893476009368896,
    'test/mae': 353.2463684082031,
    'test/mape': 5.304133892059326,
    'test/mse': 378415.96875,
    'test/norm_mae': 0.8099026679992676,
    'test/norm_mse': 3.589296579360962,
    'test/recon_loss': -1.0,
    'test/smape': 0.5212534666061401
}


NSW_SA_VIC = {
    'test/acc': 0.999920129776001,
    'test/class_loss': 0.0036400395911186934,
    'test/forecast_loss': 2.0599873065948486,
    'test/loss': 2.0603511333465576,
    'test/mae': 275.84429931640625,
    'test/mape': 4.273866176605225,
    'test/mse': 293544.09375,
    'test/norm_mae': 0.5130354762077332,
    'test/norm_mse': 2.0599663257598877,
    'test/recon_loss': -1.0,
    'test/smape': 0.5105653405189514
}


TAS_VIC = {
    'test/acc': 0.9999988675117493,
    'test/class_loss': 0.00018868206825572997,
    'test/forecast_loss': 2.00669002532959,
    'test/loss': 2.0067076683044434,
    'test/mae': 180.10763549804688,
    'test/mape': 9.2027587890625,
    'test/mse': 149156.25,
    'test/norm_mae': 0.510602593421936,
    'test/norm_mse': 2.0066702365875244,
    'test/recon_loss': -1.0,
    'test/smape': 0.4641474187374115
}


QLD_SA = {
    'test/acc': 1.0,
    'test/class_loss': 1.4576493413187563e-05,
    'test/forecast_loss': 2.115231513977051,
    'test/loss': 2.1152329444885254,
    'test/mae': 229.3065185546875,
    'test/mape': 3.9114978313446045,
    'test/mse': 177347.265625,
    'test/norm_mae': 0.6004430651664734,
    'test/norm_mse': 2.1152100563049316,
    'test/recon_loss': -1.0,
    'test/smape': 0.532358705997467
}


SA = {
    'test/acc': 1.0,
    'test/class_loss': 1.8859866031561978e-05,
    'test/forecast_loss': 1.5413442850112915,
    'test/loss': 1.5413461923599243,
    'test/mae': 147.66839599609375,
    'test/mape': 2.938582181930542,
    'test/mse': 110239.5546875,
    'test/norm_mae': 0.5044668912887573,
    'test/norm_mse': 1.5413289070129395,
    'test/recon_loss': -1.0,
    'test/smape': 0.6409849524497986
}


VIC = {
    'test/acc': 1.0,
    'test/class_loss': 1.4665401977254078e-06,
    'test/forecast_loss': 1.0325630903244019,
    'test/loss': 1.032563328742981,
    'test/mae': 302.2825012207031,
    'test/mape': 7.884873867034912,
    'test/mse': 292761.34375,
    'test/norm_mae': 0.47561830282211304,
    'test/norm_mse': 1.0325520038604736,
    'test/recon_loss': -1.0,
    'test/smape': 0.5565780401229858
}

TAS = {
    'test/acc': 1.0,
    'test/class_loss': 8.936825179262087e-05,
    'test/forecast_loss': 2.9573781490325928,
    'test/loss': 2.9573867321014404,
    'test/mae': 55.12985610961914,
    'test/mape': 6.255341529846191,
    'test/mse': 17400.708984375,
    'test/norm_mae': 0.5364540219306946,
    'test/norm_mse': 2.9573497772216797,
    'test/recon_loss': -1.0,
    'test/smape': 0.38621100783348083
}


DUNWOD_HUD_VL_LONGIL_MHK_VL_MILLWD_NYC = {
    "test/acc": 0.9999966025352478,
    "test/class_loss": 0.07700246572494507,
    "test/forecast_loss": 0.021472575142979622,
    "test/loss": 0.029172804206609726,
    "test/mae": 2654.209716796875,
    "test/mape": 3.225375175476074,
    "test/mse": 232170704.0,
    "test/norm_mae": 0.02553042396903038,
    "test/norm_mse": 0.02147235907614231,
    "test/recon_loss": -1.0,
    "test/smape": 0.7266609072685242
}

HUD_VL = {
    "test/acc": 1.0,
    "test/class_loss": 0.0,
    "test/forecast_loss": 0.013604830019176006,
    "test/loss": 0.013604830019176006,
    "test/mae": 1503.0147705078125,
    "test/mape": 1.4490935802459717,
    "test/mse": 147330368.0,
    "test/norm_mae": 0.014443130232393742,
    "test/norm_mse": 0.013604693114757538,
    "test/recon_loss": -1.0,
    "test/smape": 0.5379265546798706
}

MILLWD = {
    "test/acc": 1.0,
    "test/class_loss": 0.0,
    "test/forecast_loss": 0.01821964792907238,
    "test/loss": 0.01821964792907238,
    "test/mae": 2152.873046875,
    "test/mape": 6.771895885467529,
    "test/mse": 197615840.0,
    "test/norm_mae": 0.020671667531132698,
    "test/norm_mse": 0.018219463527202606,
    "test/recon_loss": -1.0,
    "test/smape": 1.4896466732025146
}

DUNWOD_HUD_VL_LONGIL = {
    "test/acc": 0.9999963641166687,
    "test/class_loss": 1.4901997928973287e-05,
    "test/forecast_loss": 0.01187007687985897,
    "test/loss": 0.01187157817184925,
    "test/mae": 1061.2105712890625,
    "test/mape": 0.9823232889175415,
    "test/mse": 128470880.0,
    "test/norm_mae": 0.010201247408986092,
    "test/norm_mse": 0.011869959533214569,
    "test/recon_loss": -1.0,
    "test/smape": 0.42613494396209717
}

CAPITL_CENTRL_DUNWOD_GENESE_HUD_VL_MHK_VL_MILLWD_NORTH_WEST = {
    "test/acc": 1.0,
    "test/class_loss": 2.0318542738095857e-05,
    "test/forecast_loss": 0.01120595820248127,
    "test/loss": 0.011207984760403633,
    "test/mae": 832.6868896484375,
    "test/mape": 0.9222472310066223,
    "test/mse": 121361912.0,
    "test/norm_mae": 0.008001520298421383,
    "test/norm_mse": 0.011205845512449741,
    "test/recon_loss": -1.0,
    "test/smape": 0.4380180239677429
}

CENTRL_GENESE_MHK_VL = {
    "test/acc": 0.9999998807907104,
    "test/class_loss": 0.0012411607895046473,
    "test/forecast_loss": 0.014265650883316994,
    "test/loss": 0.014389771968126297,
    "test/mae": 1583.602783203125,
    "test/mape": 1.3973913192749023,
    "test/mse": 154432240.0,
    "test/norm_mae": 0.015220343135297298,
    "test/norm_mse": 0.014265506528317928,
    "test/recon_loss": -1.0,
    "test/smape": 0.5700980424880981
}

CAPITL_DUNWOD_HUD_VL_LONGIL_MHK_VL_MILLWD = {
    "test/acc": 1.0,
    "test/class_loss": 0.0003482261090539396,
    "test/forecast_loss": 0.013380737043917179,
    "test/loss": 0.013415569439530373,
    "test/mae": 1614.340087890625,
    "test/mape": 2.150481700897217,
    "test/mse": 144900464.0,
    "test/norm_mae": 0.01551351323723793,
    "test/norm_mse": 0.013380602933466434,
    "test/recon_loss": -1.0,
    "test/smape": 0.6656721830368042
}

CAPITL_CENTRL_GENESE_HUD_VL_LONGIL_MHK_VL_MILLWD_NORTH_WEST = {
    "test/acc": 0.9999977350234985,
    "test/class_loss": 0.00011981472198385745,
    "test/forecast_loss": 0.017954139038920403,
    "test/loss": 0.017966115847229958,
    "test/mae": 1734.356689453125,
    "test/mape": 1.8922516107559204,
    "test/mse": 194373824.0,
    "test/norm_mae": 0.01666896790266037,
    "test/norm_mse": 0.017953956499695778,
    "test/recon_loss": -1.0,
    "test/smape": 0.6376346945762634
}

CENTRL = {
    "test/acc": 1.0,
    "test/class_loss": 0.0,
    "test/forecast_loss": 0.013620934449136257,
    "test/loss": 0.013620934449136257,
    "test/mae": 1521.7685546875,
    "test/mape": 0.8656660914421082,
    "test/mse": 147293408.0,
    "test/norm_mae": 0.014633851125836372,
    "test/norm_mse": 0.013620797544717789,
    "test/recon_loss": -1.0,
    "test/smape": 0.4046041667461395
}

CAPITL_CENTRL_MHK_VL = {
    "test/acc": 1.0,
    "test/class_loss": 1.652054925216362e-05,
    "test/forecast_loss": 0.011556754820048809,
    "test/loss": 0.011558408848941326,
    "test/mae": 987.944091796875,
    "test/mape": 0.7865293622016907,
    "test/mse": 125086568.0,
    "test/norm_mae": 0.009496129117906094,
    "test/norm_mse": 0.011556641198694706,
    "test/recon_loss": -1.0,
    "test/smape": 0.40760719776153564
}

NORTH = {
    "test/acc": 1.0,
    "test/class_loss": 0.0,
    "test/forecast_loss": 0.01234312355518341,
    "test/loss": 0.01234312355518341,
    "test/mae": 1078.2430419921875,
    "test/mape": 1.4331727027893066,
    "test/mse": 133822648.0,
    "test/norm_mae": 0.010355280712246895,
    "test/norm_mse": 0.012342997826635838,
    "test/recon_loss": -1.0,
    "test/smape": 0.5684621930122375
}

CAPITL_CENTRL_DUNWOD_HUD_VL_LONGIL_MHK_VL_MILLWD_NYC_WEST = {
    "test/acc": 0.9999963641166687,
    "test/class_loss": 0.003757036756724119,
    "test/forecast_loss": 0.01777811534702778,
    "test/loss": 0.018153807148337364,
    "test/mae": 2344.9326171875,
    "test/mape": 2.4068679809570312,
    "test/mse": 192257952.0,
    "test/norm_mae": 0.022553149610757828,
    "test/norm_mse": 0.017777934670448303,
    "test/recon_loss": -1.0,
    "test/smape": 0.7284622192382812
}

CENTRL_GENESE_WEST = {
    "test/acc": 0.9999991059303284,
    "test/class_loss": 2.626576679176651e-05,
    "test/forecast_loss": 0.012804306112229824,
    "test/loss": 0.012806926853954792,
    "test/mae": 1905.6837158203125,
    "test/mape": 1.2399706840515137,
    "test/mse": 138532912.0,
    "test/norm_mae": 0.01832132413983345,
    "test/norm_mse": 0.012804176658391953,
    "test/recon_loss": -1.0,
    "test/smape": 0.949256181716919
}

DUNWOD = {
    "test/acc": 1.0,
    "test/class_loss": 0.0,
    "test/forecast_loss": 0.013317253440618515,
    "test/loss": 0.013317253440618515,
    "test/mae": 1477.5126953125,
    "test/mape": 2.3620119094848633,
    "test/mse": 144338128.0,
    "test/norm_mae": 0.014192074537277222,
    "test/norm_mse": 0.01331711933016777,
    "test/recon_loss": -1.0,
    "test/smape": 0.5987970232963562
}

GENESE = {
    "test/acc": 1.0,
    "test/class_loss": 0.0,
    "test/forecast_loss": 0.013815858401358128,
    "test/loss": 0.013815858401358128,
    "test/mae": 1516.13623046875,
    "test/mape": 1.3783208131790161,
    "test/mse": 149611360.0,
    "test/norm_mae": 0.014569444581866264,
    "test/norm_mse": 0.013815720565617085,
    "test/recon_loss": -1.0,
    "test/smape": 0.573638379573822
}

MHK_VL = {
    "test/acc": 1.0,
    "test/class_loss": 0.0,
    "test/forecast_loss": 0.011379119008779526,
    "test/loss": 0.011379119008779526,
    "test/mae": 899.8667602539062,
    "test/mape": 0.9958378672599792,
    "test/mse": 123279552.0,
    "test/norm_mae": 0.008645396679639816,
    "test/norm_mse": 0.011379003524780273,
    "test/recon_loss": -1.0,
    "test/smape": 0.5659889578819275
}

LONGIL = {
    "test/acc": 1.0,
    "test/class_loss": 0.0,
    "test/forecast_loss": 0.013490378856658936,
    "test/loss": 0.013490378856658936,
    "test/mae": 1814.9346923828125,
    "test/mape": 0.833392858505249,
    "test/mse": 145718096.0,
    "test/norm_mae": 0.017462806776165962,
    "test/norm_mse": 0.013490242883563042,
    "test/recon_loss": -1.0,
    "test/smape": 0.47276484966278076
}

# DUNWOD_HUD_VL_NYC = {
#     "test/acc": 1.0,
#     "test/class_loss": 1.839381548052188e-05,
#     "test/forecast_loss": 0.011135506443679333,
#     "test/loss": 0.011137349531054497,
#     "test/mae": 854.00439453125,
#     "test/mape": 0.5639990568161011,
#     "test/mse": 120240672.0,
#     "test/norm_mae": 0.008225065656006336,
#     "test/norm_mse": 0.011135395616292953,
#     "test/recon_loss": -1.0,
#     "test/smape": 0.3226090669631958
# }


CAPITL_CENTRL_HUD_VL_MHK_VL_MILLWD_NORTH = {
    'test/acc': 0.9999864101409912,
    'test/class_loss': 0.010778824798762798,
    'test/forecast_loss': 0.014716076664626598,
    'test/loss': 0.015793945640325546,
    'test/mae': 2179.978271484375,
    'test/mape': 2.9163126945495605,
    'test/mse': 159396368.0,
    'test/norm_mae': 0.02094651758670807,
    'test/norm_mse': 0.014715931378304958,
    'test/recon_loss': -1.0,
    'test/smape': 1.063554048538208
}

CAPITL_CENTRL_HUD_VL_MHK_VL_NORTH_WEST = {
    'test/acc': 0.9999991059303284,
    'test/class_loss': 0.0015631072456017137,
    'test/forecast_loss': 0.011304641142487526,
    'test/loss': 0.01146094873547554,
    'test/mae': 968.8771362304688,
    'test/mape': 0.8284134864807129,
    'test/mse': 122387320.0,
    'test/norm_mae': 0.009311743080615997,
    'test/norm_mse': 0.011304525658488274,
    'test/recon_loss': -1.0,
    'test/smape': 0.5097997784614563
}

CAPITL_CENTRL_GENESE_HUD_VL_LONGIL_MHK_VL_N_Y_C_NORTH_WEST = {
    'test/acc': 0.9999706149101257,
    'test/class_loss': 0.11852862685918808,
    'test/forecast_loss': 0.04212141036987305,
    'test/loss': 0.05397427827119827,
    'test/mae': 4350.669921875,
    'test/mape': 3.4397659301757812,
    'test/mse': 455451456.0,
    'test/norm_mae': 0.041844308376312256,
    'test/norm_mse': 0.04212099313735962,
    'test/recon_loss': -1.0,
    'test/smape': 0.6992670297622681
}

CAPITL_MHK_VL_NORTH = {
    'test/acc': 1.0,
    'test/class_loss': 0.00019870027608703822,
    'test/forecast_loss': 0.013347466476261616,
    'test/loss': 0.013367335312068462,
    'test/mae': 1444.9425048828125,
    'test/mape': 1.5808160305023193,
    'test/mse': 144593536.0,
    'test/norm_mae': 0.013882777653634548,
    'test/norm_mse': 0.013347333297133446,
    'test/recon_loss': -1.0,
    'test/smape': 0.4336577355861664
}

DUNWOD_HUD_VL_NYC = {
    'test/acc': 1.0,
    'test/class_loss': 0.0002024953719228506,
    'test/forecast_loss': 0.013183911330997944,
    'test/loss': 0.013204153627157211,
    'test/mae': 1971.34765625,
    'test/mape': 1.5254237651824951,
    'test/mse': 142351952.0,
    'test/norm_mae': 0.018982021138072014,
    'test/norm_mse': 0.013183780014514923,
    'test/recon_loss': -1.0,
    'test/smape': 0.7006605863571167
}

capitl_central_genese_hud_vl_mhk_vl_west = {
    "test/acc": 0.9999968409538269,
    "test/class_loss": 0.00801069661974907,
    "test/forecast_loss": 0.01616775617003441,
    "test/loss": 0.016968831419944763,
    "test/mae": 1763.18505859375,
    "test/mape": 1.4713650941848755,
    "test/mse": 175001824.0,
    "test/norm_mae": 0.01694730669260025,
    "test/norm_mse": 0.016167595982551575,
    "test/recon_loss": -1.0,
    "test/smape": 0.4401610493659973
}

capitl_central_genese_hud_vl_mhk_vl_north = {
    "test/acc": 1.0,
    "test/class_loss": 0.0034146199468523264,
    "test/forecast_loss": 0.020248474553227425,
    "test/loss": 0.02058991976082325,
    "test/mae": 2349.443359375,
    "test/mape": 2.336817979812622,
    "test/mse": 219263920.0,
    "test/norm_mae": 0.022577621042728424,
    "test/norm_mse": 0.02024826779961586,
    "test/recon_loss": -1.0,
    "test/smape": 0.6717438697814941
}

capitl = {
    "test/acc": 1.0,
    "test/class_loss": 0.0,
    "test/forecast_loss": 0.012773863039910793,
    "test/loss": 0.012773863039910793,
    "test/mae": 1342.3072509765625,
    "test/mape": 1.0298148393630981,
    "test/mse": 138257024.0,
    "test/norm_mae": 0.012902302667498589,
    "test/norm_mse": 0.012773734517395496,
    "test/recon_loss": -1.0,
    "test/smape": 0.4512978196144104
}

nyc = {
    "test/acc": 1.0,
    "test/class_loss": 0.0,
    "test/forecast_loss": 0.012484879232943058,
    "test/loss": 0.012484879232943058,
    "test/mae": 1829.7938232421875,
    "test/mape": 0.3260555863380432,
    "test/mse": 133915224.0,
    "test/norm_mae": 0.01766757294535637,
    "test/norm_mse": 0.012484756298363209,
    "test/recon_loss": -1.0,
    "test/smape": 0.23400260508060455
}

capitl_central_dunwod_hud_vl_longil_mhk_vl_millwd_nyc_north = {
    "test/acc": 1.0,
    "test/class_loss": 3.023819772351999e-05,
    "test/forecast_loss": 0.01212690956890583,
    "test/loss": 0.01212993822991848,
    "test/mae": 1242.4144287109375,
    "test/mape": 1.265241026878357,
    "test/mse": 131175568.0,
    "test/norm_mae": 0.011949179694056511,
    "test/norm_mse": 0.01212678849697113,
    "test/recon_loss": -1.0,
    "test/smape": 0.5249022841453552
}

west = {
    "test/acc": 1.0,
    "test/class_loss": 0.0,
    "test/forecast_loss": 0.017124835401773453,
    "test/loss": 0.017124835401773453,
    "test/mae": 2293.335693359375,
    "test/mape": 1.2933769226074219,
    "test/mse": 185212224.0,
    "test/norm_mae": 0.022051796317100525,
    "test/norm_mse": 0.017124664038419724,
    "test/recon_loss": -1.0,
    "test/smape": 0.5141408443450928
}

dunwod_hud_vl_millwd = {
    "test/acc": 1.0,
    "test/class_loss": 0.00012867109035141766,
    "test/forecast_loss": 0.011324509978294373,
    "test/loss": 0.011337372474372387,
    "test/mae": 843.9257202148438,
    "test/mape": 1.461225986480713,
    "test/mse": 122734944.0,
    "test/norm_mae": 0.008106506429612637,
    "test/norm_mse": 0.01132439635694027,
    "test/recon_loss": -1.0,
    "test/smape": 0.7292196750640869
}



In [ ]:
def find_correct_nyiso_dict(regions_list):
    if set(regions_list) == set(['DUNWOD', 'HUD_VL', 'LONGIL', 'MHK_VL', 'MILLWD', 'NYC']):
        return DUNWOD_HUD_VL_LONGIL_MHK_VL_MILLWD_NYC
    elif set(regions_list) == set(['HUD_VL']):
        return HUD_VL
    elif set(regions_list) == set(['MILLWD']):
        return MILLWD
    elif set(regions_list) == set(['DUNWOD', 'HUD_VL', 'LONGIL']):
        return DUNWOD_HUD_VL_LONGIL
    elif set(regions_list) == set(['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD_VL', 'MHK_VL', 'MILLWD', 'NORTH', 'WEST']):
        return CAPITL_CENTRL_DUNWOD_GENESE_HUD_VL_MHK_VL_MILLWD_NORTH_WEST
    elif set(regions_list) == set(['CENTRL', 'GENESE', 'MHK_VL']):
        return CENTRL_GENESE_MHK_VL
    

In [ ]:
import shutil
shutil.make_archive('spacetimeformer', 'zip', '/kaggle/working/spacetimeformer')

In [ ]:
import os

# Path to the zip file
zip_file_path = '/kaggle/working/spacetimeformer.zip'

# Delete the file
if os.path.exists(zip_file_path):
    os.remove(zip_file_path)
    print(f"File {zip_file_path} has been deleted.")
else:
    print(f"The file {zip_file_path} does not exist.")

In [ ]:
# Correct

l = [
 ['TAS'],
 ['VIC'],
 ['SA'],
 ['QLD', 'SA'],
 ['TAS', 'VIC'],
 ['NSW', 'SA', 'VIC'],
 ['NSW', 'QLD', 'SA'],
 ['NSW', 'TAS', 'VIC'],
 ['NSW', 'QLD', 'SA', 'VIC'],
 ['NSW', 'SA', 'TAS', 'VIC']
]

CAPITL
CENTRL
MHK VL
HUD VL
LONGIL
N.Y.C.
WEST
GENESE
NORTH
DUNWOD
MILLWD
['CAPITL', 'CENTRL', 'MHK VL']
['DUNWOD', 'HUD VL', 'N.Y.C.']
['DUNWOD', 'HUD VL', 'MILLWD']
['CENTRL', 'GENESE', 'MHK VL']
['CAPITL', 'MHK VL', 'NORTH']
['DUNWOD', 'HUD VL', 'LONGIL']
['CENTRL', 'GENESE', 'WEST']
['CAPITL', 'CENTRL', 'HUD VL', 'MHK VL', 'MILLWD', 'NORTH']
['DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.']
['CAPITL', 'DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD']
['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'MHK VL', 'NORTH']
['CAPITL', 'CENTRL', 'HUD VL', 'MHK VL', 'NORTH', 'WEST']
['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'MHK VL', 'WEST']
['CAPITL', 'CENTRL', 'DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.', 'NORTH']
['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'MHK VL', 'MILLWD', 'NORTH', 'WEST']
['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'LONGIL', 'MHK VL', 'N.Y.C.', 'NORTH', 'WEST']
['CAPITL', 'CENTRL', 'DUNWOD', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.', 'WEST']
['CAPITL', 'CENTRL', 'GENESE', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'NORTH', 'WEST']
['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.', 'NORTH', 'WEST']

In [9]:
def subset_dataframe_by_year(file_path: str, time_col: str, start_year: int, end_year: int, new_file_path: str) -> None:
    """
    Reads a CSV file, filters rows where the year from time_col is between start_year and end_year (inclusive),
    and saves the resulting subset to new_file_path.
    
    Args:
        file_path (str): Path to the input CSV file.
        time_col (str): Name of the column containing date/time values.
        start_year (int): Start year (inclusive) for filtering.
        end_year (int): End year (inclusive) for filtering.
        new_file_path (str): Path to save the filtered CSV.
    """
    import pandas as pd
    
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Convert the time column to datetime
    df[time_col] = pd.to_datetime(df[time_col])
    
    # Filter rows where the year is between start_year and end_year
    subset = df[(df[time_col].dt.year >= start_year) & (df[time_col].dt.year <= end_year)]

    subset[time_col] = subset[time_col].dt.strftime("%Y-%m-%d %H:%M")
    
    # Save the subset to the new file path
    subset.to_csv(new_file_path, index=False)
    print(f"Subset of data from {start_year} to {end_year} saved to {new_file_path}")


In [11]:
subset_dataframe_by_year("C:\\Users\\vm-user\\Downloads\\All Datasets\\All Datasets\\aemo\\NSW_QLD_SA_TAS_VIC.csv", 'SETTLEMENTDATE', 2020, 2025, "C:\\Users\\vm-user\\Downloads\\dynamic\\NSW_QLD_SA_TAS_VIC.csv")

C:\Users\vm-user\AppData\Local\Temp\ipykernel_7916\3140997336.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset[time_col] = subset[time_col].dt.strftime("%Y-%m-%d %H:%M")


Subset of data from 2020 to 2025 saved to C:\Users\vm-user\Downloads\dynamic\NSW_QLD_SA_TAS_VIC.csv


In [12]:
subset_dataframe_by_year("C:\\Users\\vm-user\\Downloads\\All Datasets\\All Datasets\\nyiso\\nyiso_CAPITL_CENTRL_DUNWOD_GENESE_HUD VL_LONGIL_MHK VL_MILLWD_N.Y.C._NORTH_WEST.csv", 'Time Stamp', 2020, 2025, "C:\\Users\\vm-user\\Downloads\\dynamic\\nyiso_CAPITL_CENTRL_DUNWOD_GENESE_HUD VL_LONGIL_MHK VL_MILLWD_N.Y.C._NORTH_WEST.csv")

C:\Users\vm-user\AppData\Local\Temp\ipykernel_7916\3140997336.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset[time_col] = subset[time_col].dt.strftime("%Y-%m-%d %H:%M")


Subset of data from 2020 to 2025 saved to C:\Users\vm-user\Downloads\dynamic\nyiso_CAPITL_CENTRL_DUNWOD_GENESE_HUD VL_LONGIL_MHK VL_MILLWD_N.Y.C._NORTH_WEST.csv


In [ ]:
!python "S:\spatiotemporal-analysis\spacetimeformer-main\spacetimeformer\train.py" "spacetimeformer" "aemo-dynamic" --epochs "100" --context_points "360" --target_points "48" --run_name "nsw_qld_dynamic" --use_seasonal_decomp --l2_coeff "0.0001" --batch_size "32" --base_lr "0.0001" --learning_rate "0.0001" --d_model "50" --d_ff "100" --enc_layers "1" --dec_layers "1" --dropout_emb ".1" --dropout_ff ".3" --d_qk "25" --d_v "25" --n_heads "2" --patience "5" --data_path "C:\\Users\\vm-user\\Downloads\\dynamic\\NSW_QLD_SA_TAS_VIC.csv" --target_cols "RRP_NSW" --ignore_cols "RRP_VIC-RRP_SA-RRP_TAS-TOTALDEMAND_NSW-TOTALDEMAND_QLD-TOTALDEMAND_SA-TOTALDEMAND_TAS-TOTALDEMAND_VIC" --aemo_states "NSW-QLD"